In [6]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [7]:
using JuMP, GLPK
using DataFrames
using CSV

In [8]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:200
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [9]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

Row,hour,p_DA
,Int64,Float64


In [10]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:24])
@variable(model, delta[1:200,1:24])
@variable(model, delta_up[1:200,1:24])
@variable(model, delta_down[1:200,1:24])

# Define the objective function
@objective(model, Max, sum(sum(1/200*(scenarios[i][hour,"price"] * p_DA[hour] + 0.9*scenarios[i][hour,"price"]*delta_up[i,hour] - 1.2*scenarios[i][hour,"price"]*delta_down[i,hour]) for i in 1:200) for hour in 1:24))

# Define the constraints
@constraint(model, [hour in 1:24], p_DA[hour] <= 200)
@constraint(model, [hour in 1:24], p_DA[hour] >= 0)
@constraint(model, [i in 1:200, hour in 1:24], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:200, hour in 1:24], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:200, hour in 1:24], delta_down[i,hour] >= 0)
@constraint(model, [i in 1:200, hour in 1:24], delta_up[i,hour] >= 0)

# Solve the optimization problem
optimize!(model)

#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

No optimal solution found
